# Sentiment Analysis Prediction Insights
**Analysis of CS2 Review Predictions**

This notebook provides insights into the model's performance on the validation/test set.
We verify the results using **Plotly** with the **Asimov (二西莫夫)** color scheme.

**Focus:**
1. Overall Performance (Confusion Matrix)
2. Confidence Distribution (How sure is the model?)
3. Error Analysis (Playtime, Review Length, etc.)


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report

# Display settings
pd.set_option('display.max_colwidth', 100)


In [ ]:
# Asimov Color Scheme (二西莫夫配色)
# High contrast, sci-fi/clean aesthetic.
# Red (Negative), Blue/Cyan (Positive), Purple/Gold (Accents), Dark Background (Optional)

ASIMOV_COLORS = ['#FF004D', '#00D4FF', '#FFD700', '#F2F2F2', '#1A1A2E']
# Specific mapping for Binary Sentiment
COLOR_MAP = {
    0: '#FF004D',  # Negative (Neon Red)
    1: '#00D4FF',  # Positive (Neon Cyan)
    'Correct': '#00D4FF',
    'Incorrect': '#FF004D'
}

def apply_asimov_layout(fig, title=""):
    fig.update_layout(
        title=title,
        title_x=0.5,
        font=dict(family="Roboto, sans-serif", size=14, color="#2c3e50"),
        plot_bgcolor="#f4f6f9",
        paper_bgcolor="#ffffff",
        margin=dict(l=40, r=40, t=80, b=40),
        colorway=ASIMOV_COLORS
    )
    return fig


In [ ]:
# Load Predictions
try:
    df = pd.read_csv("cs2_full_predictions.csv")
    print(f"Loaded {len(df)} predictions.")
except FileNotFoundError:
    print("Error: 'cs2_full_predictions.csv' not found. Please upload the results file.")
    # Create dummy data for demonstration if file is missing (Optional, handled by external dummy gen)
    raise

# Ensure types
df['voted_up'] = df['voted_up'].astype(bool)
df['predicted_label'] = df['predicted_label'].astype(int)

# Add helper columns
df['actual_label'] = df['voted_up'].map({False: 0, True: 1})
df['result'] = np.where(df['actual_label'] == df['predicted_label'], 'Correct', 'Incorrect')


In [ ]:
# 1. Confusion Matrix
cm = confusion_matrix(df['actual_label'], df['predicted_label'])
z_text = [[str(y) for y in x] for x in cm]

fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Predicted Negative', 'Predicted Positive'],
    y=['Actual Negative', 'Actual Positive'],
    text=z_text,
    texttemplate="%{text}",
    textfont={"size": 20, "color": "white"},
    colorscale=[[0, '#1A1A2E'], [1, '#00D4FF']], # Dark to Blue
    showscale=True
))

apply_asimov_layout(fig_cm, "Confusion Matrix")
fig_cm.show()


In [ ]:
# 2. Model Confidence Distribution
# How confident is the model when it's right vs. wrong?

fig_conf = px.histogram(
    df, 
    x="predicted_prob", 
    color="result",
    nbins=50,
    color_discrete_map=COLOR_MAP,
    opacity=0.7,
    barmode="overlay",
    labels={"predicted_prob": "Prediction Probability (Confidence)", "count": "Count"}
)

apply_asimov_layout(fig_conf, "Confidence Distribution: Correct vs Incorrect")
fig_conf.update_traces(marker_line_width=0)
fig_conf.show()


In [ ]:
# 3. Playtime vs Accuracy
# Does the model understand veterans (high playtime) better than new players?

# Binning Playtime
bins = [0, 10, 100, 1000, 100000]
labels = ['<10h', '10-100h', '100-1000h', '>1000h']
df['playtime_group'] = pd.cut(df['author.playtime_forever'] / 60, bins=bins, labels=labels) # Convert minutes to hours

# Calculate accuracy per group
acc_by_playtime = df.groupby('playtime_group')['result'].apply(lambda x: (x == 'Correct').mean()).reset_index()
acc_by_playtime.columns = ['Playtime Group', 'Accuracy']

fig_play = px.bar(
    acc_by_playtime,
    x='Playtime Group',
    y='Accuracy',
    color='Accuracy',
    color_continuous_scale=['#FF004D', '#00D4FF'], # Red to Blue
    text_auto='.2%'
)

apply_asimov_layout(fig_play, "Model Accuracy by Player Experience (Playtime)")
fig_play.update_yaxes(range=[0, 1])
fig_play.show()


In [ ]:
# 4. Review Length vs Prediction Result
# Are longer reviews harder to classify?

df['review_length'] = df['review'].astype(str).apply(len)

fig_len = px.box(
    df,
    x="result",
    y="review_length",
    color="result",
    color_discrete_map=COLOR_MAP,
    points="outliers"
)

apply_asimov_layout(fig_len, "Review Length Distribution by Prediction Result")
fig_len.update_yaxes(type="log", title="Review Length (Log Scale)") # Log scale often helps with text length
fig_len.show()
